In [1]:
import pymongo
import pandas as pd
import numpy as np

from pymongo import MongoClient
from bson.objectid import ObjectId

import datetime

import matplotlib.pyplot as plt

from collections import defaultdict


%matplotlib inline
import json
plt.style.use('ggplot')
import seaborn as sns
from math import log10, floor
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer



from sklearn.cluster import KMeans, MiniBatchKMeans
from IPython.display import Image
from IPython.core.display import HTML

import matplotlib.pyplot as plt
import urllib.request

In [2]:
## Connect to local DB

client = MongoClient('localhost', 27017)
print ("Setup db access")
#
# Get collections from mongodb
#

db = client.test

Setup db access


In [3]:
# Find all qual_id's
qual_ids =  db.anon_student_task_responses.distinct('qual_id')




In [4]:
# print (len(qual_ids), qual_ids)

In [5]:
# Get count of qual_id with failures
qual_id_counts =  db.anon_student_task_responses.aggregate(
  [{           
    '$group': {
      '_id': {
        'qual_id': '$qual_id'
      },
      'count': {
        '$sum': 1
      }
    }
    
  }
   ]
)

In [6]:
 qual_id_counts = list(qual_id_counts)

In [7]:
qual_count_df = pd.DataFrame(qual_id_counts)

In [8]:
# print (qual_id_counts)

In [9]:
# print (qual_count_df)

In [10]:
qual_count_df = qual_count_df.sort_values(by='count', ascending=False)

In [11]:
qual_count_df.reset_index(drop=True,inplace=True)
# print (qual_count_df)

In [12]:
qual_id_fail_counts =  db.anon_student_task_responses.aggregate(
  [{           
    '$group': {
      '_id': {
        'qual_id': '$qual_id',
        'correct': 'False'             
      },
      'count': {
        '$sum': 1
      }
    }
    
  }
   ]
)

In [13]:
qual_id_fail_counts = list(qual_id_fail_counts)

In [14]:
qual_fail_count_df = pd.DataFrame(qual_id_fail_counts)

In [15]:
qual_fail_count_df.sort_values(by='count', inplace=True, ascending=False)

In [16]:
# qual_fail_count_df 

In [17]:
## Functions for turning dictionary into document

def make_string_from_list(key, elem_list):
    # Append key to each item in list
    ans = ''
    for elem in elem_list:
        ans += key + '_' + elem 
        
    
        
    

def make_string(elem, key=None, top=True):
    ans = ''
    if not elem:
        return ans
    if top:
        top = False
        top_keys = []
        for idx in range(len(elem.keys())):
            top_keys.append(True)
            
    for idx, key in enumerate(elem.keys()):
        if top_keys[idx]:
            top = True
            top_keys[idx] = False
            ans += ' '
        else:
            top = False
        #print ('ans = ', ans)
        #print (type(elem[key]))
        if type(elem[key]) is str or\
                type(elem[key]) is int:
            #print ('add value', elem[key])
            value = str(elem[key])
            #ans += key + '_' + value + ' ' + value + ' '
            ans += key + '_' + value + ' '
        elif type(elem[key]) is list:
            #print ('add list', elem[key])
            temp_elem = dict()
            for item in elem[key]:
                temp_elem[key] = item
                ans += make_string(temp_elem, top) 
        elif type(elem[key]) is dict:
            #print ('add dict', elem[key])
            for item_key in elem[key].keys():
                temp_elem = dict()
                temp_elem[item_key] = elem[key][item_key]
                ans += key + '_' + make_string(temp_elem, top)
        elif type(elem[key]) is float:
            #print ('add dict', elem[key])
            sig = 2
            value = elem[key]
            value = round(value, sig-int(
            floor(log10(abs(value))))-1)
            value = str(value)
            #ans += key + '_' + value + ' ' + value + ' '
            ans += key + '_' + value + ' '
        # ans += ' ' + key + ' '
        #print ('not handled', elem[key])
     
        
    return ans

In [18]:
def make_doc_data_set(prob_set):
    df3 = pd.DataFrame(list(prob_set))
    df3['response_doc'] = df3['response'].map(make_string)
    df3['response_doc'] = df3['response_doc'].map(lambda x: x + ' ')
    df3['response_doc'] = df3['response_doc'].map(lambda x: x.replace('/','_'))
    df3['response_doc'] = df3['response_doc'] + ' ' + df3['txt'] 
    df3['response_doc'] = df3['response_doc'].map(lambda x: x + ' ')
    df3['response_doc'] = df3['response_doc'].map(lambda x: x.replace("\n", ""))
    df3['response_doc'] = df3['response_doc'].map(lambda x: x.replace("?", " "))
    return df3
    


In [19]:


def make_k_means(data_samples, n_features=200, n_topic=30, n_top_words=10, clusters=10):
    n_samples = len(data_samples)
    print("Extracting tf-idf features ...")
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                       max_features=n_features,
                                       stop_words='english')
    t0 = time()
    tfidf = tfidf_vectorizer.fit_transform(data_samples)
    print("done in %0.3fs." % (time() - t0))
    
    # Number of clusters
    true_k = clusters
    km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1,
                             init_size=1000, batch_size=1000, random_state=62)
    print("Clustering with %s" % km)
    t0 = time()
    km.fit(tfidf)
    print("done in %0.3fs" % (time() - t0))
    return km


In [20]:
top_ten_qual_id = qual_count_df[:10]

In [21]:
this_qid = top_ten_qual_id.iloc[0]['_id']['qual_id']

In [22]:
top_ten_qual_id

,_id,count
0,{'qual_id': 'xSDXuo9OEH.bonus.OG_XxtbnEa'},22752
1,{'qual_id': 'YN4n4X5GRF.bonus.Q5XY-9tBFu'},19100
2,{'qual_id': '9wRCzK1G7F.partb.12d_Yq8M01'},18911
3,{'qual_id': 'TJxldqZMXd.partb.gqJqOrIhu_'},18390
4,{'qual_id': 'TJxldqZMXd.partb.9L_iTwGd5z'},18197
5,{'qual_id': 'TJxldqZMXd.partb.W2tLvQTbkI'},17800
6,{'qual_id': '9wRCzK1G7F.partb.SHkv5F_Yzi'},17466
7,{'qual_id': 'TJxldqZMXd.partb.hxS1eqLik1'},17191
8,{'qual_id': '9wRCzK1G7F.partb.qGGEQyiE_j'},17007
9,{'qual_id': 'kvig7fcCVc.partb.3y7UMHuR_i'},16876


In [ ]:
df_top_ten = []

In [ ]:
for row in range(len(top_ten_qual_id)):
    # print (top_ten_qual_id.iloc[row]['_id']['qual_id'], top_ten_qual_id.iloc[row]['count'])

    this_qid = top_ten_qual_id.iloc[row]['_id']['qual_id']
    count = top_ten_qual_id.iloc[row]['count']
    print ('Analysis {0} for qual_id {1} with {2} samples'.format(row, this_qid, count))


    prob_set = db.anon_student_task_responses.find({'qual_id':this_qid, 'correct': False})
    df3 = make_doc_data_set(prob_set)
    print ('Session 0 txt ', df3.iloc[0]['txt'])   
    df_top_ten.append(df3.copy())
    data_samples = df3['response_doc']
    km = make_k_means(data_samples, clusters=10)
    print ("Intertia", km.inertia_)
    df3['cluster'] = km.labels_
    print ('qual_id {0} has these clusters'.format(this_qid))
    print (df3['cluster'].value_counts())
      
    
    columns = 4
    num_images = 8
    # Look at top four clusters
    # Print 
    for cluster in (df3['cluster'].value_counts().keys())[:4]:
        plt.clf
        plt.figure(figsize=(20,10))
        print ('Cluster', cluster )

        cluster_data = df3[df3['cluster']==cluster]
        #print (cluster_data[:10])
        images = []
        for idx in range(min(num_images, df3['cluster'].value_counts()[cluster])):
            #    images.append(Image(cluster_data.iloc[idx]['screenshot_url']))
            # create a file-like object from the url
            print (cluster_data.iloc[idx]['screenshot_url'])
            try:
                f = urllib.request.urlopen(cluster_data.iloc[idx]['screenshot_url'])
            except:
                print(' could not read {0}'.format(cluster_data.iloc[idx]['screenshot_url']))
                continue
            # read the image file in a numpy array
            a = plt.imread(f)
            #plt.imshow(a)
            #plt.show()
            #print(image)
            #display(image)
            plt.subplot(min(9, df3['cluster'].value_counts()[cluster]) / columns + 1, columns, idx + 1)
            plt.imshow(a)

        plt.show()
        

    
    

Analysis 0 for qual_id xSDXuo9OEH.bonus.OG_XxtbnEa with 22752 samples
